In [191]:
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from matplotlib.legend import Legend

%matplotlib inline

In [192]:
user_name = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')
DB_NAME = 'BacterialGrowth'

# Database functions

In [243]:
user_name = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')

DB_NAME = 'BacterialGrowth'

def execute(phrase):
    """This function create a connection to the database and execute a command.
    :param phrase: str SQL sentence to execute
    :return: list of str received from the database after command execution.
    """
    try:
        cnx = mysql.connector.connect(user=user_name, password=password,host='localhost',database=DB_NAME)
        cnx.get_warnings = True
        cursor = cnx.cursor()
        cursor.execute(phrase)
        res = []
        for row in cursor:
            res.append(row)

        warnings = cursor.fetchwarnings()
        if warnings: 
            for i in range(len(warnings)):
                print("\t ** Warning - "+warnings[i][2])
        cursor.close()
        cnx.commit()
        cnx.close()
        return res
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))
        print(phrase)

def addRecord(table, args):
    """ 
    This function adds a new entry into the indicated table.

    :table: table of the DB
    :args: dictionary with the data to insert
    :return: id of the inserted record
    """
    # Insert into table
    fields, values = getInsertFieldsValues(args)
    phrase = "INSERT IGNORE INTO " +table+" "+fields+" VALUES "+values
    res = execute(phrase)
    
    # Get the name of the primary key field
    phrase = "SHOW KEYS FROM "+table+" WHERE Key_name = 'PRIMARY'"
    res = execute(phrase)
    pk = res[0][4]
    
    # Get the value of the primary key (this will return the value both if it was inserted or ignored)
    where_clause = getWhereClause(args)
    phrase = "SELECT "+pk+" FROM "+table+" "+where_clause
    res = execute(phrase)
    last_id = res[0][0]
    
    return last_id

def countRecords(table, args):
    phrase = "SELECT COUNT(*) FROM " + table
    if args:
        where_clause = getWhereClause(args)
        phrase = phrase+" "+where_clause
    res = execute(phrase)
    return res

def getAllRecords(table, **args):
    phrase = "SELECT * FROM " + table
    if args:
        where_clause = getWhereClause(args)
        phrase = phrase+" "+where_clause
    res = execute(phrase)
    return res

def getFiles(fields, args):
    where_clause = getWhereClause(args)
    fields_clause = getSelectFields(fields)
    
    phrase = "SELECT "+fields_clause+" FROM TechnicalReplicate "+where_clause
    res = execute(phrase)
    return res

def getRecords(table, fields, args):
    where_clause = getWhereClause(args)
    fields_clause = getSelectFields(fields)
    
    phrase = "SELECT "+fields_clause+" FROM "+table+" "+where_clause
    res = execute(phrase)
    return res

In [244]:
# DATABASE SUPPLEMENTARY FUNCTIONS
# =================================

def getInsertFieldsValues(args):
    fields = "("
    values = "("
    for key, val in args.items():
        fields = fields + key + ','
        values = values + "'" +str(val) + "',"
    fields = fields[:-1] + ')'
    values = values[:-1] + ')'
    return [fields, values]

def getSelectFields(args):
    clause = ""
    for field in args: 
        clause = clause + field + ", "
    clause = clause[:-2]
    return clause

def getWhereClause(args):
    if len(args) == 0:
        clause = ''
    else:
        clause = "WHERE ("
        for key, val in args.items():
            if key == 'bacteriaSpecies':
                clause = clause + key +" IN "+ str(val) + " AND "
            elif val == 'null':
                clause = clause + key + " IS NULL AND "
            elif val == 'not null':
                clause = clause + key + " IS NOT NULL AND "
            else:
                clause = clause + key + "= '" + str(val) + "' AND "
        
        clause = clause[:-5] + ')'
    
    return clause

def getJoinClause(table_from, table_to, field):
    clause = "JOIN "+table_to+" ON "+table_to+"."+field+" = "+table_from+"."+field
    return clause

def getGroupByClause(field):
    clause = "GROUP BY " + field
    return clause

def getHavingClause(agg_function, field, operator, quant, distinct=False):
    clause = "HAVING "+agg_function
    if distinct == False:
        clause = clause + "("+field+") "+operator+" "+str(quant)
    elif distinct == True:
        clause = clause +"(DISTINCT "+field+") "+operator+" "+str(quant)
    return clause

In [235]:
def getExperimentsWithBacteria(file_types, join_args, where_args):
    
    field = 'BacteriaCommunity.experimentId'
    table = 'BacteriaCommunity'
    
    phrase = "SELECT "+field+" FROM "+table+" "
        
    for arg in join_args:
        join_clause = getJoinClause(arg['table_from'], arg['table_to'], arg['field'])
        phrase = phrase+join_clause + " "
    
    where_clause = getWhereClause(where_args)
    groupby_clause = getGroupByClause(field)
    for key, val in args.items():
        if key == 'bacteriaSpecies':
            having_clause = getHavingClause('COUNT', key, '=', len(val), distinct=True)
    
    phrase = phrase +where_clause+" "+groupby_clause+" "+having_clause
    res = execute(phrase)
    
    return res

In [236]:
def getExperimentsWithMetabolites(files, where_args):
    
    field = 'TechnicalReplicate.experimentId'
    table = 'TechnicalReplicate'
    
    where_clause = getWhereClause(where_args)
    
    phrase = "SELECT DISTINCT "+field+" FROM "+table+" "+where_clause
    res = execute(phrase)
    
    return res

## 1st query: get all the data from experiments in which X bacteria species are present

```
SELECT BacteriaCommunity.experimentId
FROM BacteriaCommunity
JOIN Bacteria ON Bacteria.bacteriaId = BacteriaCommunity.bacteriaId
JOIN Experiment ON Experiment.experimentId = BacteriaCommunity.experimentId
WHERE (Bacteria.bacteriaSpecies IN ('BT', 'RI'))
GROUP BY experimentId
HAVING COUNT(DISTINCT Bacteria.bacteriaSpecies) = 2;
```

In [237]:
file_types = {'abundanceFile', 'metabolitesFile', 'phFile'}
join_args = [{'table_from': 'BacteriaCommunity', 'table_to': 'Bacteria', 'field': 'bacteriaId'}, 
            {'table_from': 'BacteriaCommunity', 'table_to': 'Experiment', 'field': 'experimentId'}]
where_args = {'bacteriaSpecies':('BT', 'RI')}

experiment_ids = getExperimentsWithBacteria(file_types, join_args, where_args)

files_dict = {}
for experiment_id in experiment_ids:
    res = getFiles(file_types, {'experimentId':experiment_id[0]})
    files_dict[experiment_id[0]] = res

In [238]:
file_types = {'abundanceFile', 'metabolitesFile', 'phFile'}
join_args = [{'table_from': 'BacteriaCommunity', 'table_to': 'Bacteria', 'field': 'bacteriaId'}, 
            {'table_from': 'BacteriaCommunity', 'table_to': 'Experiment', 'field': 'experimentId'}]

where_args = {'bacteriaSpecies':('BT', 'RI'), 'Experiment.initialPh':'3'}
where_args = {'bacteriaSpecies':('BT', 'RI'), 'Experiment.initialPh':'5'}
where_args = {'bacteriaSpecies':('BT', 'RI'), 'Experiment.experimentId':'102'}

experiment_ids = getExperimentsWithBacteria(file_types, join_args, where_args)

files_dict = {}
for experiment_id in experiment_ids:
    res = getFiles(file_types, {'experimentId':experiment_id[0]})
    files_dict[experiment_id[0]] = res

## 2nd query: get all the data from experiments in which metabolites were measures

```
SELECT DISTINCT TechnicalReplicate.experimentId 
FROM TechnicalReplicate 
WHERE (metabolitesFile IS NOT NULL)
```

In [239]:
file_types = {'abundanceFile', 'metabolitesFile', 'phFile'}
where_args = {'metabolitesFile':'not null'}

experiment_ids = getExperimentsWithMetabolites(file_types, where_args)

files_dict = {}
for experiment_id in experiment_ids:
    res = getFiles(file_types, {'experimentId':experiment_id[0]})
    files_dict[experiment_id[0]] = res

## 3rd query: get all the data from experiments in which X bacteria species are present and in which metabolites were measured

```
SELECT BacteriaCommunity.experimentId 
FROM BacteriaCommunity 
JOIN Bacteria ON Bacteria.bacteriaId = BacteriaCommunity.bacteriaId 
JOIN Experiment ON Experiment.experimentId = BacteriaCommunity.experimentId 
JOIN TechnicalReplicate ON TechnicalReplicate.experimentId = Experiment.experimentId
WHERE (bacteriaSpecies IN ('BT', 'RI') AND metabolitesFile IS NOT NULL) 
GROUP BY BacteriaCommunity.experimentId 
HAVING COUNT(DISTINCT bacteriaSpecies) = 2
```

In [240]:
file_types = {'abundanceFile', 'metabolitesFile', 'phFile'}
where_args = {'bacteriaSpecies':('BT', 'RI'), 'metabolitesFile':'not null'}
join_args = [{'table_from': 'BacteriaCommunity', 'table_to': 'Bacteria', 'field': 'bacteriaId'}, 
            {'table_from': 'BacteriaCommunity', 'table_to': 'Experiment', 'field': 'experimentId'},
            {'table_from': 'Experiment', 'table_to': 'TechnicalReplicate', 'field': 'experimentId'}]

experiment_ids = getExperimentsWithBacteria(file_types, join_args, where_args)

files_dict = {}
for experiment_id in experiment_ids:
    res = getFiles(file_types, {'experimentId':experiment_id[0]})
    files_dict[experiment_id[0]] = res

## 4th: using the previous results (3rd), keep only those in which metabolite X is measured

In [241]:
list_metabolites = ['Glucose', 'Pyruvate']

file_types = {'abundanceFile', 'metabolitesFile', 'phFile'}
where_args = {'bacteriaSpecies':('BT', 'RI'), 'metabolitesFile':'not null'}
join_args = [{'table_from': 'BacteriaCommunity', 'table_to': 'Bacteria', 'field': 'bacteriaId'}, 
            {'table_from': 'BacteriaCommunity', 'table_to': 'Experiment', 'field': 'experimentId'},
            {'table_from': 'Experiment', 'table_to': 'TechnicalReplicate', 'field': 'experimentId'}]

experiment_ids = getExperimentsWithBacteria(file_types, join_args, where_args)

files_dict = {}
for experiment_id in experiment_ids:
    res = getFiles(file_types, {'experimentId':experiment_id[0]})
    
    for i, files in enumerate(res):
        headers = pd.read_csv(files[1], sep=" ").columns
        if set(list_metabolites).issubset(set(headers.tolist())):
            
            files_dict[experiment_id[0]] = res